In [28]:
# ---------------------------------------------------------
# PROJECT: UCF Strategic Compensation Analysis
# AUTHOR: Bowen Liu
# ROLE: Data Science Consultant
# ---------------------------------------------------------

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from google.colab import files

# --- 1. DATA ENGINEERING ---
# Standardize headers
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# FILTER: Keep only Full-Time Professionals (Salaried)
if 'employment_type' in df.columns:
    df_pro = df[df['employment_type'] == 'Salaried'].copy()
else:
    df_pro = df.copy()

# Remove student placeholders
df_pro = df_pro[df_pro['name'] != 'Graduate Assistant']

# CLEAN CURRENCY
col_money = 'annual_compensation'
if col_money in df_pro.columns:
    df_pro[col_money] = df_pro[col_money].astype(str).str.replace('$', '', regex=False)
    df_pro[col_money] = df_pro[col_money].str.replace(',', '', regex=False).str.strip()
    df_pro[col_money] = pd.to_numeric(df_pro[col_money], errors='coerce')

# AGGREGATE SPLIT SALARIES
df_clean = df_pro.groupby('name').agg({
    col_money: 'sum',
    'class_title': 'first'
}).reset_index()
df_clean = df_clean.rename(columns={col_money: 'annual_salary'})

print(f"Data Pipeline Complete. Analyzing {len(df_clean)} professionals.")

# --- 2. CHART GENERATION ---

# CHART 1: INTERNAL DISTRIBUTION (MEAN vs MODE)
ucf_mean = df_clean['annual_salary'].mean()
ucf_mode = df_clean['annual_salary'].round(-3).mode()[0]

plt.figure(figsize=(12, 7))
plt.hist(df_clean['annual_salary'], bins=50, color='#FFC904', edgecolor='black', alpha=0.7)
plt.axvline(ucf_mean, color='red', linewidth=3, label=f'Average: ${ucf_mean:,.0f}')
plt.axvline(ucf_mode, color='blue', linestyle='--', linewidth=3, label=f'Typical (Mode): ${ucf_mode:,.0f}')
plt.annotate('The "Average" is skewed\nhigh by executives', xy=(ucf_mean, 100), xytext=(ucf_mean + 40000, 300),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=11)
plt.title('Internal Salary Distribution: Average vs. Typical', fontsize=16)
plt.legend()
plt.gca().xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
plt.tight_layout()
plt.savefig('chart1_internal_dist.png', dpi=300)
plt.close()

# CHART 2: TOP 15 ROLES
role_pay = df_clean.groupby('class_title')['annual_salary'].mean().sort_values(ascending=False).head(15)
plt.figure(figsize=(12, 8))
role_pay.plot(kind='barh', color='black')
plt.title('Top 15 Highest Paying Roles (Avg)', fontsize=16)
plt.gca().invert_yaxis()
plt.gca().xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
plt.tight_layout()
plt.savefig('chart2_top_roles.png', dpi=300)
plt.close()

# CHART 3: IT COMPETITIVENESS (SORTED A-Z + MARKET STARS)
market_benchmarks = {
    'Network Engineer': 82000, 'Systems Administrator': 76000,
    'Data Analyst': 72000, 'IT Support': 52000, 'Web Developer': 78000
}
keywords = [
    'IT ', 'Information Tech', 'Computer', 'Cyber', 'Network', 'Systems Admin',
    'Web Dev', 'Software', 'Database', 'Cloud', 'Security', 'Application',
    'Data Scientist', 'Data Analyst', 'Programmer', 'Solutions Architect',
    'Technical Support', 'Client Support'
]
pattern = '|'.join(keywords)
tech_df = df_clean[df_clean['class_title'].str.contains(pattern, case=False, na=False)]
# Keep titles with 3+ people
common_tech = tech_df[tech_df['class_title'].isin(tech_df['class_title'].value_counts()[tech_df['class_title'].value_counts() >= 3].index)].copy()
# Sort A-Z
sorted_titles = sorted(common_tech['class_title'].unique())
common_tech['class_title'] = pd.Categorical(common_tech['class_title'], categories=sorted_titles, ordered=True)

plt.figure(figsize=(14, 12))
common_tech.boxplot(column='annual_salary', by='class_title', vert=False, figsize=(14,12))

# Market Benchmarks with Fix for Legend Warning
label_added = False
for i, role in enumerate(sorted_titles):
    for bench_name, bench_sal in market_benchmarks.items():
        if bench_name.lower() in role.lower():
            lbl = ""
            if not label_added:
                lbl = "Market Median"
                label_added = True
            plt.plot(bench_sal, i + 1, 'r*', markersize=15, label=lbl)
            break

plt.title('Internal Pay vs. Market Benchmarks', fontsize=16)
plt.suptitle('')
plt.gca().invert_yaxis()
plt.gca().xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
if label_added:
    plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('chart3_market_comp.png', dpi=300)
plt.close()

# CHART 4: LORENZ CURVE (INEQUALITY & STATS)
sorted_sal = np.sort(df_clean['annual_salary'])
n = len(sorted_sal)
cum_people = np.arange(1, n+1) / n
cum_budget = np.cumsum(sorted_sal) / sorted_sal.sum()

# Calculations for Annotation
index_80 = int(n * 0.8)
top_20_count = n - index_80
total_budget = sorted_sal.sum()
top_20_total = sorted_sal[index_80:].sum()
top_20_share = top_20_total / total_budget

plt.figure(figsize=(10, 8))
plt.plot(cum_people, cum_budget, linewidth=3, label='Actual Distribution')
plt.plot([0,1], [0,1], 'k--', label='Perfect Equality')
plt.scatter(0.8, cum_budget[index_80], color='red', s=100, zorder=5)

# Dynamic Annotation with Stats
label_text = (f"Top 20% ({top_20_count:,.0f} Staff)\n"
              f"consumes ${top_20_total/1e6:,.1f}M\n"
              f"({top_20_share:.1%} of ${total_budget/1e6:,.1f}M Payroll)")

plt.annotate(label_text, xy=(0.8, cum_budget[index_80]), xytext=(0.2, 0.7),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=12,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.8))

plt.title('Budget Concentration (Lorenz Curve)', fontsize=16)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('chart4_inequality.png', dpi=300)
plt.close()

print("Success! All 4 charts have been generated and saved.")

Data Pipeline Complete. Analyzing 5876 professionals.
Success! All 4 charts have been generated and saved.


<Figure size 1400x1200 with 0 Axes>